In [1]:
import numpy as np
from PIL import Image
import cv2
import random
import os, sys
from pathlib import Path
import albumentations as A
from glob import glob

use_crops = False
use_hor_ver_flips = False
use_umen_x2 = True
use_umen_x4 = False
use_umen_x8 = False
use_back_images = True
use_combine_x4 = False

seed = 69
random.seed(seed)
np.random.seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)

def read_mask(filepath):
    image = Image.open(filepath).convert('L')
    image = np.array(image).astype(np.float64)
    image /= image.max()
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            image[i, j] = 0 if image[i, j] == 0.0 else 1.0
    image = image.astype(np.uint8)
    image = cv2.resize(image, (512, 512))
    return image

def read_image(filepath):
    image = Image.open(filepath)
    image = np.array(image)
    image = cv2.resize(image, (512, 512))
    return image

def horizontal_flip_and_vertical_flip(image_path, mask_path, new_images_dir='new_images',
                                     new_masks_dir='new_masks', start_image=0, repeats=2):
    image = read_image(image_path)
    mask = read_mask(mask_path)
    
    if not Path(new_images_dir).exists():
        os.mkdir(new_images_dir)
    
    if not Path(new_masks_dir).exists():
        os.mkdir(new_masks_dir)
    
    hor_flip = A.Compose([A.HorizontalFlip(p=1.0)])
    vert_flip = A.Compose([A.VerticalFlip(p=1.0)])
    both = A.Compose([A.VerticalFlip(p=1.0), A.HorizontalFlip(1.0)])
    
    for repeat in range(repeats):
    
#         cv2.imwrite(os.path.join(new_images_dir, str(start_image) + '.png'), image)
#         cv2.imwrite(os.path.join(new_masks_dir, str(start_image) + '.png'), mask*255)
#         start_image += 1

        image1 = hor_flip(image=image)['image']
        mask1 = hor_flip(image=mask)['image']
        cv2.imwrite(os.path.join(new_images_dir, str(start_image) + '.png'), image1)
        cv2.imwrite(os.path.join(new_masks_dir, str(start_image) + '.png'), mask1*255)
        start_image += 1

        image2 = vert_flip(image=image)['image']
        mask2 = vert_flip(image=mask)['image']
        cv2.imwrite(os.path.join(new_images_dir, str(start_image) + '.png'), image2)
        cv2.imwrite(os.path.join(new_masks_dir, str(start_image) + '.png'), mask2*255)
        start_image += 1

        image3 = both(image=image)['image']
        mask3 = both(image=mask)['image']
        cv2.imwrite(os.path.join(new_images_dir, str(start_image) + '.png'), image3)
        cv2.imwrite(os.path.join(new_masks_dir, str(start_image) + '.png'), mask3*255)
        start_image += 1


def random_crop(image_path, mask_path, new_images_dir='new_images', 
                new_masks_dir='new_masks', num_crops=50, start_image=0):
    image = read_image(image_path)
    mask = read_mask(mask_path)
    
    if not Path(new_images_dir).exists():
        os.mkdir(new_images_dir)
        
    if not Path(new_masks_dir).exists():
        os.mkdir(new_masks_dir)
    
    cv2.imwrite(os.path.join(new_images_dir, str(start_image) + '.png'), image)
    cv2.imwrite(os.path.join(new_masks_dir, str(start_image) + '.png'), mask)
    
    cnt = 1
    while cnt < num_crops:
        x1, y1 = random.randint(0, 256), random.randint(0, 256)
#         x2, y2 = random.randint(64, 511), random.randint(64, 511)
#         while (x2 - x1) < 64 or (y2 - y1) < 64:
#             x2, y2 = random.randint(64, 511), random.randint(64, 511)
        x2, y2 = x1 + 255, y1 + 255
        
        new_image = image[y1:y2+1, x1:x2+1, :]
        new_mask = mask[y1:y2+1, x1:x2+1] * 255
        
        new_image_path = os.path.join(new_images_dir, str(start_image+cnt)+'.png')
        new_mask_path = os.path.join(new_masks_dir, str(start_image+cnt)+'.png')
        
        cv2.imwrite(new_image_path, new_image)
        cv2.imwrite(new_mask_path, new_mask)
        cnt += 1
        
def make_umen_images_xx(image_path, mask_path, koef, new_images_dir='new_images',
                     new_masks_dir='new_masks', start_image=0):
    image = read_image(image_path)
    mask = read_mask(mask_path)
    
    new_image = np.zeros((512, 512, 3))
    new_mask = np.zeros((512, 512))
    
    image = cv2.resize(image, (512//koef, 512//koef))
    mask = cv2.resize(mask, (512//koef, 512//koef))
    
    new_image[256-256//koef:256+256//koef, 256-256//koef:256+256//koef, :] = image
    new_mask[256-256//koef:256+256//koef, 256-256//koef:256+256//koef] = mask
    
    cv2.imwrite(os.path.join(new_images_dir, str(start_image) + '.png'), new_image)
    cv2.imwrite(os.path.join(new_masks_dir, str(start_image) + '.png'), new_mask)
    
    
def combine_x4_images(image_path, mask_path, images_path, masks_path,
                      new_images_dir='new_images',new_masks_dir='new_masks',
                      start_image=0):
    image = read_image(image_path)
    mask = read_mask(mask_path)
    
    image = cv2.resize(image, (256, 256))
    mask = cv2.resize(mask, (256, 256))
    
    additional_images_path = random.sample(images_path, 3)
    additional_masks_path = [os.path.join(masks_path, img_path.split('/')[-1])
                             for img_path in images_path]
    
    add_image1 = read_image(additional_images_path[0])
    add_image2 = read_image(additional_images_path[1])
    add_image3 = read_image(additional_images_path[2])
    
    add_image1 = cv2.resize(add_image1, (256, 256))
    add_image2 = cv2.resize(add_image2, (256, 256))
    add_image3 = cv2.resize(add_image3, (256, 256))
    
    add_mask1 = read_mask(additional_masks_path[0])
    add_mask2 = read_mask(additional_masks_path[1])
    add_mask3 = read_mask(additional_masks_path[2])
    
    add_mask1 = cv2.resize(add_mask1, (256, 256))
    add_mask2 = cv2.resize(add_mask2, (256, 256))
    add_mask3 = cv2.resize(add_mask3, (256, 256))
    
    new_image = np.zeros((512, 512, 3))
    new_mask = np.zeros((512, 512))
    
    new_image[:256, :256, :] = image
    new_image[:256, 256:, :] = add_image1
    new_image[256:, :256, :] = add_image2
    new_image[256:, 256:, :] = add_image3
    
    new_mask[:256, :256] = mask
    new_mask[:256, 256:] = add_mask1
    new_mask[256:, :256] = add_mask2
    new_mask[256:, 256:] = add_mask3
    
    cv2.imwrite(os.path.join(new_images_dir, str(start_image) + '.png'), new_image)
    cv2.imwrite(os.path.join(new_masks_dir, str(start_image) + '.png'), new_mask)
    
def pad_with_background_from_another_images(image_path, mask_path, backs_path, koef, new_images_dir='new_images',
                     new_masks_dir='new_masks', start_image=0):
    image = read_image(image_path)
    mask = read_mask(mask_path)
    
    random_back_name = random.sample(backs_path, 1)[0]
    
    new_image = read_image(random_back_name)
    new_mask = np.zeros((512, 512))
    image = cv2.resize(image, (512//koef, 512//koef))
    mask = cv2.resize(mask, (512//koef, 512//koef))
    
    new_image[256-256//koef:256+256//koef, 256-256//koef:256+256//koef, :] = image
    new_mask[256-256//koef:256+256//koef, 256-256//koef:256+256//koef] = mask
    
    cv2.imwrite(os.path.join(new_images_dir, str(start_image) + '.png'), new_image)
    cv2.imwrite(os.path.join(new_masks_dir, str(start_image) + '.png'), new_mask)
    
def mixup_with_background_from_another_images(image_path, mask_path, backs_path, koef, new_images_dir='new_images',
                     new_masks_dir='new_masks', start_image=0):
    image = read_image(image_path)
    mask = read_mask(mask_path)
    
    random_back_name = random.sample(backs_path, 1)[0]
    
    new_image = read_image(random_back_name)
    new_image = new_image * 0.25 + image * 0.75
    
    cv2.imwrite(os.path.join(new_images_dir, str(start_image) + '.png'), new_image)
    cv2.imwrite(os.path.join(new_masks_dir, str(start_image) + '.png'), mask)
    
def create_cropped_dataset(images_pathes, masks_pathes, new_images_dir,
                           new_masks_dir, start_image=0):
    transform = A.Compose(
        [
            A.CropNonEmptyMaskIfExists(width=256, height=256),
        ]
    )
    
    for image_path, mask_path in zip(images_pathes, masks_pathes):
        image = read_image(image_path)
        mask = read_mask(mask_path)
        for _ in range(1):
            t = transform(image=image, mask=mask)
            cv2.imwrite(os.path.join(new_masks_dir, str(start_image)+'.png'), t['mask'])
            cv2.imwrite(os.path.join(new_images_dir, str(start_image)+'.png'), t['image'])


In [2]:
# images_path = os.listdir(os.path.join('images', '0'))
# masks_path = os.listdir(os.path.join('annotation', 'semantic'))
images_path = os.listdir('original_images')
masks_path = os.listdir('original_masks')
backs_path = glob('background/*')

images_path

['71.png',
 '128.png',
 '21.png',
 '75.png',
 '102.png',
 '125.png',
 '109.png',
 '152.png',
 '106.png',
 'images-2.png',
 '25.png',
 '124.png',
 '114.png',
 '51.png',
 '70.png',
 '11.png',
 '45.png',
 '18.png',
 'images15.png',
 '91.png',
 'images12.png',
 '150.png',
 '28.png',
 '143.png',
 '40.png',
 '83.png',
 'images35.png',
 'images24.png',
 '8.png',
 '59.png',
 '101.png',
 'images36.png',
 '123.png',
 'images16.png',
 'images-3.png',
 '126.png',
 '24.png',
 '67.png',
 'images33.png',
 'images19.png',
 '7.png',
 'images3.png',
 '56.png',
 '50.png',
 '2.png',
 '108.png',
 '94.png',
 '117.png',
 '60.png',
 '57.png',
 'images44.png',
 '66.png',
 'images43.png',
 '47.png',
 '92.png',
 '105.png',
 '81.png',
 '23.png',
 '27.png',
 '120.png',
 '64.png',
 '145.png',
 '147.png',
 '3.png',
 '131.png',
 'images29.png',
 '30.png',
 '127.png',
 '19.png',
 '43.png',
 '32.png',
 '16.png',
 '90.png',
 'images.png',
 'images34.png',
 '77.png',
 '118.png',
 'images20.png',
 '46.png',
 '136.png',
 '

In [3]:
num_crops = 3 if use_crops else 0
repeats = 1
start_image = 0

for i in range(len(images_path)):
#     image_path = os.path.join('images', '0', images_path[i])
#     mask_path = os.path.join('annotation', 'semantic', masks_path[i])
    image_path = os.path.join('original_images', images_path[i])
    mask_path = os.path.join('original_masks', masks_path[i])

    random_crop(image_path, mask_path, num_crops=num_crops, start_image=i*(num_crops+1))

In [4]:
# horizontal_flip_and_vertical_flip(os.path.join('images', '0', images_path[0]), 
#                                   os.path.join('annotation', 'semantic', masks_path[0]), 
#                 start_image=len(images_path)*num_crops, repeats=repeats)

# horizontal_flip_and_vertical_flip(os.path.join('images', '0', images_path[1]), 
#                                   os.path.join('annotation', 'semantic', masks_path[1]), 
#                 start_image=len(images_path)*num_crops+4*repeats, repeats=repeats)

In [5]:
if use_hor_ver_flips:
    for i in range(len(images_path)):
        image_path = os.path.join('original_images', images_path[i])
        mask_path = os.path.join('original_masks', masks_path[i])

        horizontal_flip_and_vertical_flip(image_path, mask_path, 
            start_image=len(images_path)*(num_crops+1)+i*3, repeats=repeats)

In [6]:
if use_umen_x2:
    for i in range(len(images_path)):
        image_path = os.path.join('original_images', images_path[i])
        mask_path = os.path.join('original_masks', masks_path[i])
        if use_hor_ver_flips:
            start_image = len(images_path) * (num_crops+1) + len(images_path) * 3 + i
        else:
            start_image = len(images_path) * (num_crops+1) + i
            
        make_umen_images_xx(image_path, mask_path, 2, start_image=start_image)
        

In [7]:
if use_umen_x4:
    for i in range(len(images_path)):
        image_path = os.path.join('original_images', images_path[i])
        mask_path = os.path.join('original_masks', masks_path[i])
        if use_hor_ver_flips:
            start_image = len(images_path) * (num_crops+1) + len(images_path) * 3 + i
        else:
            start_image = len(images_path) * (num_crops+1) + i
            
        if use_umen_x2:
            start_image += len(images_path)
            
        make_umen_images_xx(image_path, mask_path, 4, start_image=start_image)
        

In [8]:
if use_back_images:
    for i in range(len(images_path)):
        image_path = os.path.join('original_images', images_path[i])
        mask_path = os.path.join('original_masks', masks_path[i])
        if use_hor_ver_flips:
            start_image = len(images_path) * (num_crops+1) + len(images_path) * 3 + i
        else:
            start_image = len(images_path) * (num_crops+1) + i
            
        if use_umen_x2:
            start_image += len(images_path)
        
        if use_umen_x4:
            start_image += len(images_path)
            
        mixup_with_background_from_another_images(image_path, mask_path, backs_path, 2, start_image=start_image)

In [9]:
if use_combine_x4:
    for i in range(len(images_path)):
        image_path = os.path.join('original_images', images_path[i])
        mask_path = os.path.join('original_masks', masks_path[i])
        if use_hor_ver_flips:
            start_image = len(images_path) * (num_crops+1) + len(images_path) * 3 + i
        else:
            start_image = len(images_path) * (num_crops+1) + i
            
        if use_umen_x2:
            start_image += len(images_path)
        
        if use_umen_x4:
            start_image += len(images_path)
        
        if use_back_images:
            start_image += len(images_path)
            
        combine_x4_images(image_path, mask_path, glob('original_images/*'), 
                          'original_masks', start_image=start_image)